# Exploratory analysis using Xorbits over NYC taxi dataset
Xorbits is a powerful tool for exploring and analyzing large datasets. One of the classic
datasets for demonstrating the capabilities of Xorbits is the NYC taxi dataset, which
contains records of taxi rides in the city from 2009 to 2022. In this blog, we will
explore how to use Xorbits to do some initial exploration of the NYC taxi dataset and get
a sense of what kind of insights we might be able to gain from the data.

## Software versions
- Xorbits==0.1.0
- plotly==5.11.0

In [ ]:
# Install dependencies
%pip install xorbits==0.1.0 plotly==5.11.0 pyarrow

## Datasets
- [TLC Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
- [NYC Taxi Zones](https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc)

Download the taxi zone lookup table and the taxi zone geojson:

In [ ]:
%%bash
wget -q https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv
wget -q https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc\?method\=export\&format\=GeoJSON -O taxi_zones.geojson

You can start with yellow taxi trip records of a month:

In [ ]:
%%bash
wget -q https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-12.parquet

You can also use records of a year:

In [ ]:
%%bash
for i in {1..12}
do
    echo "downloading taxi records of $i/2021"
    wget -q https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-$(printf "%02d" $i).parquet
done

## Initialization
The first step is to initialize Xorbits:

In [ ]:
import xorbits

# Initialize Xorbits in the local environment.
xorbits.init()

## Data loading
The second step is to load the data into an Xorbits DataFrame. This can be done using the
`read_parquet()` function, which allows us to specify the location of the parquet file and
any additional options we want to use while reading the data.


In the case of the NYC taxi dataset, here is an example of how we could do this using
Xorbits:

In [ ]:
import datetime
import json
import xorbits.pandas as pd

trips = pd.read_parquet("yellow_tripdata_2021-*.parquet")
# Remove outliers
trips = trips[(trips['tpep_pickup_datetime'] >= datetime.datetime(2021, 1, 1)) & (trips['tpep_pickup_datetime'] <= datetime.datetime(2021, 12, 31))]

taxi_zones = pd.read_csv('taxi+_zone_lookup.csv')

with open('taxi_zones.geojson') as fd:
    geojson = json.load(fd)

Once we have the data loaded into a DataFrame, we might want to get a sense of the
overall structure of the data by looking at the number of rows and columns, the data types
of each column, and the first few rows of the data. We can do this using the shape, dtypes,
and head() attributes, respectively:

In [ ]:
trips.shape

In [ ]:
trips.dtypes

In [ ]:
trips.head()

We can also use the `describe()` method to get a summary of the statistical properties of
each numerical column in the dataset.

In [ ]:
trips.describe()

## Time series analysis
One way to analyze the NYC taxi dataset is to look at how the number of rides varies over time.
We can do this by creating a new column in the DataFrame that represents the pick-up date of
each ride, and then use the `groupby` method to group the data by month or year and compute the
count of rides for each group:

In [ ]:
trips['PU_date'] = trips['tpep_pickup_datetime'].dt.date
count = trips.groupby('PU_date', as_index=False).agg(count=('VendorID', 'count'))
count

We can then use a library like plotly to visualize the time series data:

In [ ]:
import plotly.express as px

b = px.bar(count.to_pandas(), x='PU_date', y='count')
b.show()

## Spatial analysis
Another way to analyze the NYC taxi dataset is to look at patterns in the spatial distribution of
rides. Taking Manhattan as an example, we firstly filter the dataframe by pick-up location ID:

In [ ]:
manhattan_zones = taxi_zones[taxi_zones['Borough'] == 'Manhattan']['LocationID']
manhattan_trips = trips[trips['PULocationID'].isin(manhattan_zones)]
manhattan_trips

Then use the `groupby` method to group the data by pick-up location ID and compute the count of
rides for each group:

In [ ]:
gb_pu_location = manhattan_trips.groupby(['PULocationID'], as_index=False).agg(count=('VendorID', 'count')).to_pandas()

We can then use a library like plotly to visualize the spatial distribution of rides:

In [ ]:
m = px.choropleth(
    gb_pu_location,
    geojson=geojson,
    locations='PULocationID',
    featureidkey='properties.location_id',
    color='count',
    color_continuous_scale="Viridis",
    range_color=(0, gb_pu_location['count'].max()),
    labels={'count':'trips count'}
)
m.update_geos(fitbounds="locations", visible=False)
m.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
m.show()